In [3]:
import os 
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch

mesh_decoder_files_in_current_folder = glob.glob('/home/ralbe/DALS/mesh_autodecoder/models/MeshDecoderTrainer_2025-11-06_12-00*.ckpt')
convnet_files_in_current_folder = glob.glob('ConvNet*ckpt')
# List to hold dicts for each file
mesh_decoder_data_list = []

for file in mesh_decoder_files_in_current_folder:
    loaded_file = torch.load(file, map_location='cpu')
    print(loaded_file['hparams'])
    print(loaded_file.keys())
    row = {
        'filename': os.path.basename(file),
        'train_path': loaded_file['train_data_path'],
        'val_path': loaded_file['val_data_path'],
        'databank_size': loaded_file['latent_vectors'].weight.shape[0],
        'databank_latent_dim': loaded_file['latent_vectors'].weight.shape[1],
        'template_shape': loaded_file['template'].verts_packed().shape,
        'backbone_mode': loaded_file['decoder_mode'],
        'best_loss': loaded_file['best_loss'],
        'best_epoch': loaded_file['best_epoch'],
        'weight_norm_loss': loaded_file['hparams']['weight_norm_loss'],
        'weight_normal_loss': loaded_file['hparams']['weight_normal_loss'],
        'weight_quality_loss': loaded_file['hparams']['weight_quality_loss'],
        'weight_laplacian_loss': loaded_file['hparams']['weight_laplacian_loss'],
        'weight_edge_loss': loaded_file['hparams']['weight_edge_loss'],
    }
    mesh_decoder_data_list.append(row)

mesh_decoder_df = pd.DataFrame(mesh_decoder_data_list)




{'latent_features': 128, 'steps': 1, 'subdivide': False, 'hidden_features': [724, 724, 362], 'concat_latent_at': [], 'template_subdiv': 3, 'decoder_mode': 'gcnn', 'encoding': 'none', 'encoding_order': 8, 'normalization': 'layer', 'rotate_template': True, 'remesh_every': 0, 'remesh_every_edge_length_ratio': 1.0, 'remesh_at': [], 'remesh_at_edge_length_ratio': [], 'num_epochs': 9999, 'batch_size': 8, 'weight_normal_loss': 0.01, 'weight_norm_loss': 0.001, 'weight_edge_loss': 0.01, 'weight_laplacian_loss': 0.01, 'weight_quality_loss': 0.001, 'use_kendall_gal': True, 'num_mesh_samples': 2500, 'train_batch_size': 8, 'learning_rate_net': 0.002, 'learning_rate_lv': 0.002, 'lr_reduce_factor': 0.5, 'lr_reduce_patience': 50, 'lr_reduce_min_lr': 1e-05, 'save_shapes_every': 0, 'save_shapes_dir': './saved_shapes', 'no_checkpoints': False, 'checkpoint_postfix': '', 'checkpoint_dir': '.', 'random_seed': 1337, 'resume_from': None, 'profiling': False}
dict_keys(['decoder_state_dict', 'optimizer_state_di

In [9]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)  # This line will show full column contents
print('\n\n')
df = mesh_decoder_df[['filename', 'databank_latent_dim', 'weight_edge_loss','best_epoch','best_loss']].sort_values(['databank_latent_dim', 'weight_edge_loss']).copy()
df['best_loss'] = df['best_loss'].apply(lambda x: f'{x:.2e}')
print(df)

print(df.iloc[-1])




                                      filename  databank_latent_dim  \
0  MeshDecoderTrainer_2025-11-06_12-05-56.ckpt                  128   

   weight_edge_loss  best_epoch best_loss  
0              0.01         236  7.02e-03  
filename               MeshDecoderTrainer_2025-11-06_12-05-56.ckpt
databank_latent_dim                                            128
weight_edge_loss                                              0.01
best_epoch                                                     236
best_loss                                                 7.02e-03
Name: 0, dtype: object


In [20]:
convnet_data_list = []

for file2 in convnet_files_in_current_folder:
    loaded_file = torch.load(file2, map_location='cpu')
    row = {
        'model_name': loaded_file['hparams']['model'],
        'best_dice': loaded_file['best_metric'],
        'ckpt_filepath': os.path.abspath(file2)
    }
    convnet_data_list.append(row)

convnet_df = pd.DataFrame(convnet_data_list)



In [21]:
print(convnet_df)
print('\n\n')
print(convnet_df.iloc[-1])

  model_name  best_dice  \
0    DynUNet   0.928963   
1    ResUNet   0.894558   
2  SEGRESNET   0.925203   
3  SYNVNET3D   0.917074   
4       UNET   0.896039   
5      UNETR   0.895987   
6       VNet   0.935717   

                                                                             ckpt_filepath  
0    /home/ralbe/DALS/mesh_autodecoder/models/ConvNetTrainer_DynUNet_2025-10-15_15-36.ckpt  
1    /home/ralbe/DALS/mesh_autodecoder/models/ConvNetTrainer_ResUNet_2025-10-15_15-36.ckpt  
2  /home/ralbe/DALS/mesh_autodecoder/models/ConvNetTrainer_SEGRESNET_2025-10-15_18-37.ckpt  
3  /home/ralbe/DALS/mesh_autodecoder/models/ConvNetTrainer_SYNVNET3D_2025-10-15_17-41.ckpt  
4       /home/ralbe/DALS/mesh_autodecoder/models/ConvNetTrainer_UNET_2025-10-15_16-23.ckpt  
5      /home/ralbe/DALS/mesh_autodecoder/models/ConvNetTrainer_UNETR_2025-10-15_15-36.ckpt  
6       /home/ralbe/DALS/mesh_autodecoder/models/ConvNetTrainer_VNet_2025-10-15_15-36.ckpt  



model_name                          